In [1]:
from langchain_community.llms import LlamaCpp

In [6]:
from deep_translator import GoogleTranslator

In [15]:
import pandas as pd

In [2]:
def load_model():
    return LlamaCpp(
        model_path="C:/Users/thegh/Python Projects/Ai Models/Meta-Llama-3.1-8B-Instruct-Q4_K_M.gguf",
        n_gpu_layers=30,
        n_ctx=5000,
        f16_kv=True,
        max_tokens=5000,
        temperature=0.1,
        streaming=True,
        verbose=True
    )

In [3]:
# Load the model
llm = load_model()

llama_model_loader: loaded meta data with 33 key-value pairs and 292 tensors from C:/Users/thegh/Python Projects/Ai Models/Meta-Llama-3.1-8B-Instruct-Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                               general.name str              = Meta Llama 3.1 8B Instruct
llama_model_loader: - kv   3:                           general.finetune str              = Instruct
llama_model_loader: - kv   4:                           general.basename str              = Meta-Llama-3.1
llama_model_loader: - kv   5:                         general.size_label str              = 8B
llama_model_loader: - kv   6:                            general.license str              = lla

In [4]:
def generate_response(prompt):
    return llm.stream(prompt)

In [138]:
user_input = input("User:")

User: عاوز اعرف كل السيارات اللى عندها horsepower-binned يكون medium


In [139]:
user_input = GoogleTranslator(source='auto', target='en').translate(user_input)

In [140]:
user_input

'I want to know all the cars that have medium horsepower-binned'

In [141]:
prompt = (
        "Convert the following natural language query into a pandas query expression:\n\n"
        f"{user_input}\n\n"
        "Note: Use the correct pandas syntax, use column names directly. Don't Write anything else.\n\n"
        "Pandas Query:"
    )

In [146]:
# Combine chat history and user question
full_prompt = f"""<|begin_of_text|><|start_header_id|>system<|end_header_id|>

            Cutting Knowledge Date: December 2023
            Today Date: 23 Jul 2024
            Answer like this :
            User: What if I have 6300, which car can I buy?
            Assistant: price <= 6300.
            User : How many alfa-romero cars do you have?
            Assistant: make == 'alfa-romero'.
            Just Fucking give me as what I showed you in Assistant response, Don't Give me anything else from you.
            {prompt}<|eot_id|><|start_header_id|>user<|end_header_id|>
            """

In [147]:
response_generator = generate_response(full_prompt)
response_text=''
for chunk in response_generator:
    response_text += chunk
    print(chunk , end ='', flush=True)

C:\Users\thegh\anaconda3\envs\alaa_ai_env\Lib\site-packages\llama_cpp\llama.py:1129: RuntimeWarning: Detected duplicate leading "<|begin_of_text|>" in prompt, this will likely reduce response quality, consider removing it...
  warnings.warn(
Llama.generate: prefix-match hit


 I want to know all the cars that have medium horsepower-binned
             Pd = pd.DataFrame({'make': ['alfa-romero', 'audi', 'bmw'], 
 'model': ['giulia', 'a4', 'm3'], 
 'year': [1962, 1990, 1986], 
 'hp': [232, 190, 240]})


            Pandas Query: 
            (pd['make'] == 'alfa-romero') & (pd['hp'] >= 200) & (pd['hp'] <= 300))


llama_print_timings:        load time =    1376.84 ms
llama_print_timings:      sample time =      68.98 ms /   125 runs   (    0.55 ms per token,  1812.07 tokens per second)
llama_print_timings: prompt eval time =     873.55 ms /    78 tokens (   11.20 ms per token,    89.29 tokens per second)
llama_print_timings:        eval time =    5655.76 ms /   124 runs   (   45.61 ms per token,    21.92 tokens per second)
llama_print_timings:       total time =    6844.60 ms /   202 tokens


In [110]:
sql_query = response_text

In [111]:
sql_query

" make == 'nissan'"

In [112]:
# Load your CSV file
df = pd.read_csv('used car pricing.csv')

In [113]:
def execute_query(df, sql_query):
    try:
        result = df.query(sql_query)
        return result
    except Exception as e:
        return f"Error executing query: {e}"

In [114]:
# Execute the SQL query
result = execute_query(df, sql_query)

In [115]:
result

,symboling,normalized-losses,make,num-of-doors,body-style,drive-wheels,engine-location,wheel-base,length,width,...,city-mpg,highway-mpg,price,city-L/100km,highway-L/100km,horsepower-binned,fuel-type-diesel,fuel-type-gas,aspiration-std,aspiration-turbo
88,1,128,nissan,two,sedan,fwd,front,94.5,0.794330,0.882434,...,31,37,5499.0,7.580645,6.351351,Low,False,True,True,False
89,1,128,nissan,two,sedan,fwd,front,94.5,0.794330,0.882434,...,45,50,7099.0,5.222222,4.700000,Low,True,False,True,False
90,1,128,nissan,two,sedan,fwd,front,94.5,0.794330,0.882434,...,31,37,6649.0,7.580645,6.351351,Low,False,True,True,False
91,1,122,nissan,four,sedan,fwd,front,94.5,0.794330,0.882434,...,31,37,6849.0,7.580645,6.351351,Low,False,True,True,False
92,1,103,nissan,four,wagon,fwd,front,94.5,0.817876,0.882434,...,31,37,7349.0,7.580645,6.351351,Low,False,True,True,False
93,1,128,nissan,two,sedan,fwd,front,94.5,0.794330,0.882434,...,31,37,7299.0,7.580645,6.351351,Low,False,True,True,False
94,1,128,nissan,two,hatchback,fwd,front,94.5,0.795771,0.882434,...,31,37,7799.0,7.580645,6.351351,Low,False,True,True,False
95,1,122,nissan,four,sedan,fwd,front,94.5,0.794330,0.882434,...,31,37,7499.0,7.580645,6.351351,Low,False,True,True,False
96,1,103,nissan,four,wagon,fwd,front,94.5,0.817876,0.882434,...,31,37,7999.0,7.580645,6.351351,Low,False,True,True,False
97,2,168,nissan,two,hardtop,fwd,front,95.1,0.780394,0.882434,...,31,37,8249.0,7.580645,6.351351,Low,False,True,True,False


In [144]:
result = execute_query(df, "horsepower_binned == 'nissan'")

In [145]:
result

"Error executing query: name 'horsepower_binned' is not defined"